In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
import datasets
import math
import numpy as np

pl.seed_everything(42)

text = datasets.load_dataset('tiny_shakespeare')["train"][0]["text"]

In [2]:
device = "cpu"
block_size = 64
fixed_epoch_size = block_size * 200

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, text):
        super().__init__()
        vocab = sorted(set(text))
        self.vocab = vocab
        self.stoi = { ch: i for i, ch in enumerate(vocab) }
        self.itos = { i: ch for i, ch in enumerate(vocab) }
        self.data = torch.tensor([self.stoi[ch] for ch in text], dtype=torch.long).to(device)

    def __len__(self):
        return min(fixed_epoch_size, self.data.size(0) - block_size - 1)

    def __getitem__(self, i):
        i = np.random.randint(0, self.data.size(0) - block_size - 1)
        end = i + block_size
        return self.data[i:end], self.data[i + 1:end + 1]

dataset = MyDataset(text)

In [3]:
# adapted from https://github.com/karpathy/minGPT/blob/master/mingpt/model.py
# and https://github.com/karpathy/nanoGPT/blob/master/model.py
# defaults for gpt-mini
class MinGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim=192, num_heads=6, num_layers=6, dropout=0.1):
        super().__init__()
        self.transformer = nn.ModuleDict(dict(
            token_emb = nn.Embedding(vocab_size, embed_dim),
            pos_emb = nn.Embedding(block_size, embed_dim),
            drop = nn.Dropout(dropout),
            layers = nn.Sequential(*[DecoderLayer(embed_dim, num_heads, dropout) for _ in range(num_layers)]),
            norm = nn.LayerNorm(embed_dim),
        ))
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, x):
        pos = torch.arange(0, x.size(1), dtype=torch.long).unsqueeze(0).to(device)
        x = self.transformer.token_emb(x) + self.transformer.pos_emb(pos)
        x = self.transformer.drop(x)
        x = self.transformer.layers(x)
        x = self.transformer.norm(x)
        x = self.lm_head(x)
        return x

    @torch.no_grad()
    def generate(self, input_ids, max_new_tokens, top_k=10):
        for _ in range(max_new_tokens):
            out = self(input_ids[:, -block_size:])
            logits = out[:, -1, :]
            v, _ = torch.topk(logits, top_k)
            logits[logits < v[:, [-1]]] = -float("Inf")
            step_res = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            # auto-regression
            input_ids = torch.cat((input_ids, step_res), dim=1)

        return input_ids

class DecoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout):
        super().__init__()
        self.ln1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout, batch_first=True)
        self.ln2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, 4 * embed_dim),
            nn.GELU(approximate="tanh"),
            nn.Linear(4 * embed_dim, embed_dim),
            nn.Dropout(dropout)
        )
        self.register_buffer("mask", ~torch.tril(torch.ones(block_size, block_size)).to(bool).to(device))

    def forward(self, x):
        B, T, _ = x.shape
        x = self.ln1(x)
        x = x + self.attn(x, x, x, need_weights=False, attn_mask=self.mask[:T, :T])[0]
        x = x + self.mlp(self.ln2(x))
        return x

class Model(pl.LightningModule):
    def __init__(self, vocab_size):
        super().__init__()
        self.model = MinGPT(vocab_size, embed_dim=192, num_heads=4, num_layers=2, dropout=0.1)
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self.model(x)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), ignore_index=-1)
        #self.log("train_loss", loss)
        return loss
    
    def training_epoch_end(self, outs):
        with torch.no_grad():
            x = torch.tensor([dataset.stoi[s] for s in "O God, O God!"], dtype=torch.long)[None,...].to(device)
            y = self.model.generate(x, 100)[0]
            print("".join([dataset.itos[int(i)] for i in y]))
    
    def configure_optimizers(self):
        optim = torch.optim.AdamW(self.parameters(), lr=0.005)
        sched = torch.optim.lr_scheduler.ExponentialLR(optim, gamma=0.999, last_epoch=-1)
        return [optim], [sched]

model = Model(len(dataset.vocab))
trainer = pl.Trainer(gradient_clip_val=1.0, max_epochs=20, enable_progress_bar=True, log_every_n_steps=50, accelerator="gpu" if device == "cuda" else "cpu")
trainer.fit(model, torch.utils.data.DataLoader(dataset, batch_size=64, num_workers=0))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params
---------------------------------
0 | model | MinGPT | 927 K 
---------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/Users/cztomsik/miniconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0: 100%|██████████| 200/200 [00:22<00:00,  8.79it/s, loss=1.91, v_num=45]O God, O God!


SICING MISIO:
O your tizenter mined not not suffrow, in sin see,
Areentled wher peell withy where
Epoch 1: 100%|██████████| 200/200 [00:22<00:00,  9.04it/s, loss=1.73, v_num=45]O God, O God! have had brothen there's these arm ouncame,
But on thin dare hath.

LEONTES:
I sparrown: my lipt ag
Epoch 2: 100%|██████████| 200/200 [00:22<00:00,  9.02it/s, loss=1.65, v_num=45]O God, O God!

KING RICHARD II:
He woundred it.

DUKE VINCENTIO:
I'll she'll thou wond not he heates stands: him,
Epoch 3: 100%|██████████| 200/200 [00:45<00:00,  4.42it/s, loss=1.6, v_num=45] O God, O God! meo, sir; thou borning. You art it
Farwicking thereit into slaud thus he sword
Toes in
thereof brac
Epoch 4: 100%|██████████| 200/200 [00:31<00:00,  6.26it/s, loss=1.58, v_num=45]O God, O God!

CAPULET:
Thou that we will I have not for the hadness?

MISTRESCE:
And you wound, thet malicers an
Epoch 5: 100%|██████████| 200/200 [

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 200/200 [00:26<00:00,  7.54it/s, loss=1.44, v_num=45]


In [10]:
y = model.to(device).model.generate(torch.tensor([dataset.stoi[s] for s in "ROMEO:"]).unsqueeze(0).to(device), 650)
print("".join([dataset.itos[int(i)] for i in y[0]]))

ROMEO:
And nurse! he hast you do't not.

CLAUDIO:
Now thy love to the sanniser, sir; for
yieldst you had night our sop,
We shall be nainighting to a more.

KING RICHARD II:
We are consenfully find my help in house,
It not in my fail word, are diviney,
Then in the witnege.

MARCIUEUS:
Never speak it.
I do't will thou didst recommend,
The world ission so woth my hope
That teet, butter more than heavands to my woman,
That all well no love, a fimers. Lord, baspy
And not
Intemper-living of looken to mine honour,
With mine el answorn his lodging bed mine
Of both of despite this
Time success, beyolden frest; and her made to mar so;
But, and he died of him
